# Use Case

As a physician, I want to calculate a patient's cardiovascular risk using the Framingham calculator - http://reference.medscape.com/calculator/framingham-coronary-risk-ldl

Several peices of data are needed to perform this calculation. We will use FHIR to pull the data we need from the patient's health record.

In [1]:
!pip3 install requests

You are using pip version 18.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


#### First, let's import and define some variables

In [ ]:
import datetime
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
import requests
import json

# Define the FHIR Endpoint
fhir_url = "https://api.logicahealth.org/test231/open/"

#### Next, let's get demographical information about the patient

In [8]:
# Perform a GET on patient with id of 'SMART-1627321'

# Print the first and last name of the patient

# Print the patient's date of birth

# Print patient's gender
gender = None
# Calcuate patient's age using rdelta method of the dateutil module

# print the years from the rdelta object
age = None

Patient's Name =  Frank Taylor
DOB =  1953-09-05
Gender =  male
Patient's age =  66


#### Next is to get the LDL and HDL values
LOINC codes:  
&nbsp;&nbsp;&nbsp;LDL: 18262-6  
&nbsp;&nbsp;&nbsp;HDL: 2085-9

In [20]:
#LDL
LDL_val = None

# HDL 
HDL_val = None

86.0 mg/dL
36.0 mg/dL


#### Now we want the systolic and diastolic heart pressures
LOINC code: 55284-4

In [12]:
sys = None
dia = None


Systolic: 107 mmHg
Diastolic: 60 mmHg


#### Does the patient have diabetes?
Of course, there are many types of diabetes. Normally, you would call a "ValueSet" resource to get all possible diabetes codes, but to keep things simple, we'll just use the code for Type 2 diabetes.  

SNOMED: 44054006

In [13]:
diabetes = False


They have diabetes


#### Does the patient smoke?
Again, there are many codes for smoking. First, use the code for "Tobacco smoking status" to get all Observation resources that define the patient's smoking status. Then filter by SNOMED code for "Occasional tobacco smoker".

LOINC "Tobacco smoking status": 72166-2  
SNOMED "Occasional tobacco smoker": 428041000124106

In [16]:
smoking = False


They smoke


### Coronary Risk Calculator

In [17]:
points = 0

if gender == "male":
    if age <= 34:
        points += -1
    if age >= 40 and age <= 44:
        points += 1
    if age >= 45 and age <= 49:
        points += 2
    if age >= 50 and age <= 54:
        points += 3
    if age >= 55 and age <= 59:
        points += 4
    if age >= 60 and age <= 64:
        points += 5
    if age >= 65 and age <= 69:
        points += 6
    if age >= 70:
        points += 7
    if LDL_val < 100:
        points += -3
    if LDL_val >= 160 and LDL_val <= 189:
        points += 1
    if LDL_val >= 190:
        points += 2
    if HDL_val < 35:
        points += 2
    if HDL_val >= 35 and LDL_val <= 44:
        points += 1
    if HDL_val >= 60:
        points += -1
    if sys <=129 and dia >= 85 and dia <= 89:
        points += 1
    if sys >= 130 and sys <=139 and dia < 90:
        points += 1
    if sys <= 139 and dia >= 90 and dia <= 99:
        points += 2
    if sys >= 140 and sys <=159 and dia < 100:
        points += 2
    if sys > 159 or dia > 99:
        points += 3
    if diabetes == True:
        points += 2
    if smoking == True:
        points += 2

In [18]:

if gender == "female":
    if age <= 34:
        points += -9
    if age >= 35 and age <= 39:
        points += -4
    if age >= 45 and age <= 49:
        points += 3
    if age >= 50 and age <= 54:
        points += 6
    if age >= 55 and age <= 59:
        points += 7
    if age >= 60:
        points += 8
    if LDL_val < 100:
        points += -2
    if LDL_val >= 160:
        points += 2
    if HDL_val < 35:
        points += 5
    if HDL_val >= 35 and HDL_val <= 44:
        points += 2
    if HDL_val >= 45 and HDL_val <= 49:
        points += 1
    if HDL_val >= 60:
        points += -2
    if sys < 120 and dia < 80:
        points += -3
    if sys <=159 and dia >=90 and dia < 100:
        points += 2
    if sys > 159 or dia > 99:
        points += 3
    if diabetes == True:
        points += 4
    if smoking == True:
        points += 2

In [19]:
male_results = {0:3,1:4,2:4,3:6,4:7,5:9,6:11,7:14,8:18,9:22,10:27,11:33,12:40,13:47}
female_results = {0:2,1:2,2:3,3:3,4:4,5:5,6:6,7:7,8:8,9:9,10:11,11:13,12:15,13:17,14:20,15:24,16:27}

if points >=14 and gender == "male":
    risk = ">= 56%"
elif points >=17 and gender == "female":
    risk = ">= 32%"
elif gender == "male":
    percent = male_results[points]
    risk = str(percent)+"%"
else:
    percent = female_results[points]
    risk = str(percent)+"%"
print("Total number of points: " + str(points))
print("Risk of Heart Disease: " + risk)

Total number of points: 7
Risk of Heart Disease: 14%
